<a href="https://colab.research.google.com/github/sidhant-guliani/Deep-learning-projects/blob/master/addition_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os

import random
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers, Sequential

from sklearn.model_selection import train_test_split



# New section

In [ ]:
MAX_LIMIT = 50000
MAX_DIGITS = 3
MAX_LEN_QUEST = MAX_DIGITS + MAX_DIGITS + 1 
REVERSE = False
# +1 for the "+" sign

In [ ]:
class char_DataGen:
  """ Given set of characters
  + one_hot_encoder 
  + one_hot_decoder reverses converts the 
  """
  def __init__(self, char):
    self.char = sorted(set(char))
    self.char_table = dict((c, i) for i, c in enumerate(self.char))
    self.rev_char_table = dict((i, c) for i, c in enumerate(self.char))

  def one_hot_encoder(self, q, ans= False):
    """ one hot encofer for a string
    arguments:
    q: string that you want to hot encode
    """ 
    x = np.zeros((len(q), len(self.char)))
    for i, c in enumerate(q):
      x[i, self.char_table[c]] = 1
    return x

  def decoder(self, x, calc_argmax = True):
    """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
    if calc_argmax:
            x = x.argmax(axis=-1)
    return "".join(self.rev_char_table[x] for x in x)

step1: We are generating queston and the expected answers. make sure you follow the rules while creating the datasets. Rules:
1. 

step2: 

In [ ]:
questions = []
answers = []
seen = set()

chars = "0123456789+ "

while len(questions) < MAX_LIMIT:
  f = lambda: int("".join(np.random.choice(list("0123456789"))
                  for i in range(np.random.randint(1,MAX_DIGITS+1))))
  
  a, b = f(), f()
  # skip anycase which is already in the list
  #hence we are storing the combination in seen
  combo = tuple(sorted((a,b)))
  if combo in seen:
    continue
  seen.add(combo)
  quest = "{}+{}".format(a, b)
  # padding for the question
  questn = quest + " "*(MAX_LEN_QUEST - len(quest))
  ans = str(a+ b)
  # the answer should be of length 4 so applying padding
  ans += " "*(1+MAX_DIGITS - len(ans))

  questions.append(questn)
  answers.append(ans)

print("question shape", len(questions))

question shape 50000


In [ ]:
x_data = np.zeros((MAX_LIMIT, MAX_LEN_QUEST, len(chars)))
y_data = np.zeros((MAX_LIMIT, MAX_DIGITS+1, len(chars)))

# initiating an object for the class
datagen = char_DataGen(chars)

for i, q in enumerate(questions):
  x_data[i] = datagen.one_hot_encoder(q)

for i, q in enumerate(answers):
  y_data[i] = datagen.one_hot_encoder(q)

In [ ]:
print(x_data.shape)
print(y_data.shape)

(50000, 7, 12)
(50000, 4, 12)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
#
model.add(layers.LSTM(128, input_shape=(MAX_LEN_QUEST, len(chars))))
model.add (layers.RepeatVector(MAX_DIGITS+1))
model.add(layers.LSTM(128,return_sequences= True))
model.add(layers.Dense(len(chars), activation = "softmax"))
model.compile(loss= "categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train,
          y_train,
          batch_size= 32,
          epochs = 15,
          validation_data= (x_val, y_val))


Epoch 1/15
1250/1250 [==============================] - 20s 16ms/step - loss: 1.7907 - accuracy: 0.3475 - val_loss: 1.6266 - val_accuracy: 0.3976
Epoch 2/15
1250/1250 [==============================] - 20s 16ms/step - loss: 1.4396 - accuracy: 0.4596 - val_loss: 1.2971 - val_accuracy: 0.5119
Epoch 3/15
1250/1250 [==============================] - 21s 17ms/step - loss: 1.2188 - accuracy: 0.5407 - val_loss: 1.1562 - val_accuracy: 0.5602
Epoch 4/15
1250/1250 [==============================] - 21s 17ms/step - loss: 1.0769 - accuracy: 0.5898 - val_loss: 1.0306 - val_accuracy: 0.5998
Epoch 5/15
1250/1250 [==============================] - 23s 18ms/step - loss: 0.9584 - accuracy: 0.6364 - val_loss: 0.9166 - val_accuracy: 0.6528
Epoch 6/15
1250/1250 [==============================] - 21s 16ms/step - loss: 0.8665 - accuracy: 0.6712 - val_loss: 0.8409 - val_accuracy: 0.6749
Epoch 7/15
1250/1250 [==============================] - 21s 16ms/step - loss: 0.7388 - accuracy: 0.7180 - val_loss: 0.6679 -

In [ ]:
# selecting 10 samples from the validation set so taht we can visualise the errors
for i in range(10):
  ind = np.random.randint(0, len(x_val))
  rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
  preds = np.argmax(model.predict(rowx), axis=-1)
  q = datagen.decoder(rowx[0])
  a = datagen.decoder(rowy[0])
  guess = datagen.decoder(preds[0], calc_argmax= False)
  print("Q", q[::-1] if REVERSE else q, end=" ")
  print("T", a, end=" ")
  if a == guess:
      print("☑ " + guess)
  else:
      print("☒ " + guess)

Q 464+23  T 487  ☑ 487 
Q 396+821 T 1217 ☑ 1217
Q 360+340 T 700  ☒ 701 
Q 575+9   T 584  ☑ 584 
Q 6+72    T 78   ☑ 78  
Q 21+44   T 65   ☑ 65  
Q 9+4     T 13   ☑ 13  
Q 45+97   T 142  ☑ 142 
Q 3+810   T 813  ☑ 813 
Q 731+607 T 1338 ☑ 1338


In [ ]:
print(np.argmax(model.predict(rowx), axis = -1))
print(model.predict(rowx)[0])

[[10  9 10  0]]
[[7.43903376e-12 5.94585700e-11 3.83026553e-08 2.34667095e-11
  8.31297357e-12 2.33038046e-12 6.76173188e-14 2.79715530e-12
  5.49028698e-08 9.33744665e-03 9.90556479e-01 1.06015796e-04]
 [7.28135271e-13 1.32997929e-13 1.20561727e-10 5.59233175e-14
  1.22611206e-15 3.75090457e-15 2.78114933e-14 4.91843233e-10
  1.97083704e-04 8.96950245e-01 1.02849334e-01 3.33244384e-06]
 [3.89028454e-09 3.65312351e-15 1.16237953e-08 4.21112382e-15
  1.69217711e-17 9.92142016e-18 3.25851415e-15 2.02662175e-11
  4.87718594e-08 4.38124378e-04 9.95148718e-01 4.41303663e-03]
 [9.99590099e-01 6.23125141e-14 8.94967656e-09 3.03646234e-15
  8.72050965e-16 1.55507771e-17 6.07662072e-16 1.37787318e-12
  1.41058298e-09 1.08745271e-05 3.51555471e-04 4.74809058e-05]]


In [ ]:
for i, q in enumerate(questions[:10]):
  print(questions[i], answers[i])
  aa = datagen.one_hot_encoder(q)
  bb = datagen.one_hot_encoder(answers[i])
  print(aa)
  print(bb)
  print(" ")

41+76   117 
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
52+60   112 
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 
2+70    72  
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 

In [ ]:
ind = np.random.randint(0, len(x_val))
rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
preds = np.argmax(model.predict(rowx), axis=-1)
q = datagen.decoder(rowx[0])
a = datagen.decoder(rowy[0])
guess = datagen.decoder(preds[0], calc_argmax= False)
print("Q", q[::-1] if REVERSE else q, end=" ")
print("T", a, end=" ")
if a == guess:
    print("☑ " + guess)
else:
    print("☒ " + guess)

Q 481+819 T 1102 ☑ 1102


In [ ]:
print(x_val[np.array([ind])])

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]]


In [ ]:
print(rowx[0])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


'481+819'